In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import mediapipe as mp
import os

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import os

In [ ]:
mp_holistic = mp.solutions.holistic # Modelo Holístico
mp_desenho = mp.solutions.drawing_utils # Utilidades para desenhos.

In [ ]:
def mediapipe_detection(imagem, modelo):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB) # Conversão de cor, de BGR to RGB.
    imagem.flags.writeable = False                   # Desabilita a escrita na imagem
    results = modelo.process(imagem)                 # Faz as previsões.
    imagem.flags.writeable = True                    # Abilita a escrita na imagem.
    imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2BGR) # Conversão de cor, de RGB to BGR.
    return imagem, results


def desenhar_pontos(imagem, resultados):
    mp_desenho.draw_landmarks(imagem, resultados.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                              mp_desenho.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), # Cor/Estilo dos pontos.
                              mp_desenho.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1) # Cor/Estilo das conexões.
                              ) # Desenhar as conexões e os pontos no contorno da FACE
    
    mp_desenho.draw_landmarks(imagem, resultados.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_desenho.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_desenho.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2),
                              ) # Desenhar as conexões e os pontos no Tronco
    
    mp_desenho.draw_landmarks(imagem, resultados.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_desenho.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_desenho.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              ) # Desenhar as conexões e os pontos na mão esquerda
    
    mp_desenho.draw_landmarks(imagem, resultados.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_desenho.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_desenho.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                              ) # Desenhar as conexões e os pontos na mão direita
    return imagem


# Função para extrair as variáveis espaciais dos landmarks de cada frame:
def extract_points(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose, face, left_hand, right_hand])

In [ ]:
DATA_PATH = os.path.join('MP_DATA')

# Classes/Ações que queremos detectar: (Podemos expandir isso!!)
actions = np.array([chr(i) for i in range(ord('A'), ord('I') + 1)])
 
number_of_videos = 60 # Número de sequências ou vídeos que iremos coletar os frames (Pode ser modificado!)
sequence_length = 180 # Para cada vídeo, iremos coletar 30 frames. (Pode ser modificado!)

### Preprocessamento:

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []

for action in actions:
    for sequence in range(number_of_videos):
        janela = []
        for numero_frame in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(numero_frame)))
            janela.append(res)
        sequences.append(janela)
        labels.append(label_map[action])

In [ ]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=42)

### Treinamento:

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu')) # return_sequences False, pois não iremos retornar as sequencias para a camada Fully connected.
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))